In [1]:
import faiss
import numpy as np
import pandas as pd
from rich.progress import track

In [2]:
# hdf_path = "../data/tables/20240621/clamp.h5"
hdf_path = "../data/tables/20250110/specdiff.h5"
df = pd.read_hdf(hdf_path)
df.head()

ValueError: Dataset(s) incompatible with Pandas data types, not table, or no datasets found in HDF5 file.

In [ ]:
# new_df = df.copy()  # avoid modifying the original DataFrame


# def rename_index(index):
#     if index.endswith(".mid"):
#         return f"{index[:-4]}_t00s00"  # remove '.mid' and append '_t00s00'
#     return index  # if not ending with '.mid', keep it as is


# new_df.index = [rename_index(idx) for idx in new_df.index]
# new_df.head()
# new_df.to_hdf("specdiff.h5", key="dataset")

In [3]:
def normalize_vector(vec):
    norm = np.linalg.norm(vec)
    return vec / norm if norm != 0 else vec


df["normed_embeddings"] = df["embeddings"].apply(normalize_vector)
df.head()

,embeddings,normed_embeddings
20240117-064-2b_0044-0052.mid,NaN,NaN
20240117-064-2b_0059-0067.mid,NaN,NaN
20240117-064-2b_0112-0119.mid,NaN,NaN
20240117-064-2b_0142-0149.mid,NaN,NaN
20240117-064-2b_0074-0082.mid,NaN,NaN


In [4]:
index = faiss.IndexFlatIP(768)
embedding_array = np.array(df["normed_embeddings"].tolist(), dtype=np.float32)
index.add(embedding_array)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (11356,) + inhomogeneous part.

In [ ]:
k = 5
distances, indices = index.search(
    np.array(
        [df.loc["20240117-064-2b_0044-0052.mid", "normed_embeddings"]],
        dtype=np.float32,
    ),
    k=k,
)

In [12]:
# Print the most similar documents
for i, index in enumerate(indices[0]):
    distance = distances[0][i]
    print(f"Nearest neighbor {i+1}: {df.iloc[index, :].name}, Distance {distance:.05f}")

Nearest neighbor 1: 20240117-064-2b_0044-0052.mid, Distance 1.00000
Nearest neighbor 2: 20240117-064-2b_0037-0044.mid, Distance 0.97732
Nearest neighbor 3: 20240117-064-2b_0014-0022.mid, Distance 0.96528
Nearest neighbor 4: 20240117-064-2b_0007-0014.mid, Distance 0.96513
Nearest neighbor 5: 20240117-064-2b_0067-0074.mid, Distance 0.95847
